## Test for wps_resolve_rules ProcessErrors
The usual test `run_wps_process` takes the default input value when the given value is invalid, and since every input in `wps_resolve_rules` has a default, the test always passes. Running a live instance of the process here returns the expected errors to test against. 

In [1]:
from birdy import WPSClient
import json
import os
from wps_tools.testing import get_target_url
from pkg_resources import resource_filename

In [2]:
while os.path.basename(os.getcwd()) != "sandpiper":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("sandpiper")
print(f"Using sandpiper on {url}")
sandpiper = WPSClient(url)

In [4]:
def test_wps_resolve_rules_err(
    csv,
    connection_string,
    geoserver="https://docker-dev03.pcic.uvic.ca/geoserver/bc_regions/ows"
):
    with open(csv, "r") as csv_file:
        output = sandpiper.resolve_rules(
            csv_content=csv_file.read(),
            geoserver=geoserver,
            connection_string=connection_string,
        )
    assert "Process error" in output.errors[0].text


In [5]:
# NBVAL_IGNORE_OUTPUT
try:
    with open('credentials.json') as f:
        data = json.load(f)
        connection_string = data['connection_string']
except Exception as e:
    print("You must run `make credentials` to create the required file")
    raise e

if connection_string == 'not_set':
    print("You must set the `connection_string` param in the `credentials.json` in order to run this demo")

else:
    rules = resource_filename("tests", "data/rules_small.csv")
    bad_rules = resource_filename("tests", "data/bad_rules.csv")
    
    test_wps_resolve_rules_err(rules, "bad_connection_string")
    test_wps_resolve_rules_err(bad_rules, connection_string)
    test_wps_resolve_rules_err(rules, connection_string, geoserver="bad_geoserver")

 owslib.wps.WPSException : {'code': 'NoApplicableCode', 'locator': 'None', 'text': 'Process error: ArgumentError: Could not parse rfc1738 URL from string bad_connection_string'}
 owslib.wps.WPSException : {'code': 'NoApplicableCode', 'locator': 'None', 'text': 'Process error: UnboundLocalError: local variable var referenced before assignment'}
 owslib.wps.WPSException : {'code': 'NoApplicableCode', 'locator': 'None', 'text': 'Process error: MissingSchema: Invalid URL bad_geoserver : No schema supplied. Perhaps you meant http://bad_geoserver?'}
